In [52]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
from sklearn import preprocessing
from sklearn.metrics import precision_score

train_in = pd.read_csv('train_in.csv', header=None)
train_out = pd.read_csv('train_out.csv', header=None)
test_in = pd.read_csv('test_in.csv', header=None)
test_out = pd.read_csv('test_out.csv', header=None)

In [53]:
# insert a column of one in the train set for the bias
train_in.insert(0, 'w0', 1)
test_in.insert(0, 'w0', 1)

# generate Weights randomly
rng = default_rng(seed=42)
vals = rng.standard_normal(size=(257, 10)) # standard Gaussian
W = preprocessing.MaxAbsScaler().fit_transform(vals) # uniform (on [-1, 1])

delta_w = np.full((257, 10), 0.00)

# encode the desired label to binary colomns
encoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
trainout = pd.DataFrame(encoder.fit_transform(train_out).toarray())
testout = pd.DataFrame(encoder.fit_transform(test_out).toarray())

In [63]:
# train the network on train set
# -> set a better converge conditon
for i in range(50):
    for index, x in train_in.iterrows():
        net = np.dot(x, W)           # linear regression
        out = 1 / (1 + np.exp(-net)) # logistic regression
        # generate the delta w using neuron learning rule
        for i in range(trainout.shape[1]): 
            w_tran = 0.1 * x * (trainout.iloc[index].iloc[i] - out[i]) * out[i] * (1 - out[i])
            w_tran = w_tran.values
            w_tran = w_tran.reshape(1,-1).T
            delta_w[:, i] = w_tran[:, 0]
        W += delta_w

In [61]:
# precision_score(test_set.target, predicted, average='macro')

In [64]:
# calculate the accuracy
traincorrect = 0
testcorrect = 0
for i, x in train_in.iterrows():
    pre = np.full((1, 9), 0.00)
    phi = 1 / (1 + np.exp(-np.dot(x, W)))
    digit = np.argmax(phi)
    output = np.insert(pre, digit, 1)
    if np.array_equiv(trainout.values[i], output):
        traincorrect += 1
correctr_train = traincorrect / len(train_in) * 100

for j, x in test_in.iterrows():
    pre = np.full((1, 9), 0.00)
    phi = 1 / (1 + np.exp(-np.dot(x, W)))
    digit = np.argmax(phi)
    output = np.insert(pre, digit, 1)
    if np.array_equiv(testout.values[j], output):
        testcorrect += 1
correctr_test = testcorrect / len(test_in) * 100

print('Train percentage correct: %f' % correctr_train)
print('Test percentage correct: %f' % correctr_test)

Train percentage correct: 93.497364
Test percentage correct: 85.000000
